In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [307]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51259_2024-04-16~2025-04-16 (1).xlsx')
ka_df = pd.read_excel('/Users/seanyu_mac/Downloads/20250416165128.xlsx')

In [309]:
fs_df['구분'] = '친한스토어'

In [311]:
fs_df = fs_df[['구분', '결제 번호', '상품 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '정상가',
       '판매가', '쿠폰 할인', '포인트 할인', '적립금', '정산가', '수령인명', '수령인 연락처', '구매자명',
       '구매자 ID', '구매자 연락처', '주소', '우편번호', '배송 메모', '주문 확인일', '발주 요청일',
       '배송 완료일', '구매 확정일', '배송택배사', '송장 번호', '배송비 정책', '배송비', '도서산간 배송비',
       '취소 완료일', '판매자 취소일', '반품 요청일', '수거 요청일', '수거 완료일', '반품 완료일']]

In [313]:
fs_df.columns

Index(['구분', '결제 번호', '상품 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량',
       '정상가', '판매가', '쿠폰 할인', '포인트 할인', '적립금', '정산가', '수령인명', '수령인 연락처',
       '구매자명', '구매자 ID', '구매자 연락처', '주소', '우편번호', '배송 메모', '주문 확인일', '발주 요청일',
       '배송 완료일', '구매 확정일', '배송택배사', '송장 번호', '배송비 정책', '배송비', '도서산간 배송비',
       '취소 완료일', '판매자 취소일', '반품 요청일', '수거 요청일', '수거 완료일', '반품 완료일'],
      dtype='object')

In [315]:
fs_df['주문 상태'].unique().tolist()

['배송중', '배송완료']

In [317]:
ka_df['주문상태'].unique().tolist()

['202 배송 요청', '304 배송 중', '305 배송 완료', '204 결제 취소 완료']

In [319]:
ka_df = ka_df[ka_df['주문상태'].isin(['202 배송 요청', '304 배송 중', '305 배송 완료'])]

In [321]:
ka_df['주문상태'].unique().tolist()

['202 배송 요청', '304 배송 중', '305 배송 완료']

In [323]:
cancel_df = ka_df[ka_df['주문상태'].isin(['204 결제 취소 완료'])]

In [325]:
ka_df['결제 번호'] = ka_df['결제번호']
ka_df['상품 번호'] = ka_df['채널상품번호']
ka_df['주문 번호'] = ka_df['주문번호']
ka_df['결제 완료일'] = ka_df['주문일']
ka_df['주문 상태'] = ka_df['주문상태']
ka_df['상품명'] = ka_df['상품명']
ka_df['옵션명'] = ka_df['옵션']
ka_df['수량'] = ka_df['수량']
ka_df['정상가'] = ka_df['상품금액']
ka_df['판매가'] = ka_df['상품금액'] + ka_df['옵션금액'] - ka_df['판매자할인금액']
ka_df['쿠폰 할인'] = 0
ka_df['포인트 할인'] = 0
ka_df['적립금'] = 0
ka_df['정산가'] = ka_df['정산기준금액']
ka_df['수령인명'] = '***'
ka_df['수령인 연락처'] = '***'
ka_df['구매자명'] = '***'
ka_df['구매자 ID'] = '***'
ka_df['구매자 연락처'] = '***'
ka_df['주소'] = '***'
ka_df['우편번호'] = '***'
ka_df['배송 메모'] = '***'
ka_df['주문 확인일'] = ka_df['주문일']
ka_df['발주 요청일'] = ka_df['송장입력일']
ka_df['배송 완료일'] = ka_df['배송예정일']
ka_df['구매 확정일'] = ''
ka_df['배송택배사'] = ka_df['택배사']
ka_df['송장 번호'] = ka_df['송장번호']
ka_df['배송비 정책'] = ka_df['배송비지불방법']
ka_df['배송비'] = ka_df['기본배송비 금액']
ka_df['도서산간 배송비'] = ka_df['도서산간 추가 배송비 금액']
ka_df['취소 완료일'] = ka_df['취소완료일']
ka_df['판매자 취소일'] = ka_df['취소완료일']
ka_df['반품 요청일'] = ''
ka_df['수거 요청일'] = ''
ka_df['수거 완료일'] = ''
ka_df['반품 완료일'] = ka_df['환불완료일']
ka_df['구분'] = '톡스토어'

In [327]:
ka_df = ka_df[fs_df.columns]

In [329]:
temp_df = pd.concat([fs_df[fs_df.columns], ka_df[fs_df.columns]], ignore_index=True)

In [331]:
temp_df.rename(columns={'정산가': '매출'}, inplace=True)
temp_df = temp_df[temp_df['주문 상태'].isin(['결제완료', '배송중', '202 배송 요청', '304 배송 중'])]

In [333]:
temp_df['결제 완료일'] = pd.to_datetime(temp_df['결제 완료일'])
df2 = temp_df.groupby(pd.Grouper(key='결제 완료일', freq='D'))[['결제 번호']].size()
df2

결제 완료일
2025-04-14    10
2025-04-15     9
2025-04-16     7
Freq: D, dtype: int64

In [335]:
temp_df['주문일'] = temp_df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
temp_df['박스수량'] = temp_df['옵션명'].apply(lambda x: int(x.split('_')[1].split('박스')[0].strip()))
pivot_df = pd.pivot_table(temp_df, index=['주문일', '옵션명'], values=['매출', '박스수량'], aggfunc=['sum'], fill_value=0, margins=True, margins_name='총합')

pivot_df2 = pd.pivot_table(temp_df, index=['주문일', '옵션명'], values=['결제 번호'], aggfunc=['count'], fill_value=0, margins=True, margins_name='총합')
pivot_df2 = pd.DataFrame(pivot_df2)
col1 = pivot_df2[('count', '결제 번호')]

result_df = pd.DataFrame(pivot_df)
result_df[('sum', '주문수량')] = col1

result_df.to_excel(f'/Users/seanyu_mac/Downloads/샤인약사님_매출_{datetime.now()}.xlsx')
result_df

sum          
                                             매출 박스수량 주문수량
주문일        옵션명                                           
2025-04-14 선택: 파이토레드큐민(정제)_12박스(45%할인)   726000   24    2
           선택: 파이토레드큐민(정제)_3박스(25%할인)    247500    6    2
           선택: 파이토레드큐민(정제)_6박스(36%할인)    211200    6    1
           선택: 퓨어레드큐민(분말)_3박스(46%할인)      79000    3    1
           선택: 퓨어레드큐민(분말)_6박스(53%할인)     278000    6    1
           파이토레드큐민(정제)_6박스(36%할인)        211200    6    1
           퓨어레드큐민(분말)_3박스(46%할인)          79000    3    1
           퓨어레드큐민(분말)_6박스(53%할인)         139000    6    1
2025-04-15 선택: 파이토레드큐민(정제)_12박스(45%할인)   363000   12    1
           선택: 파이토레드큐민(정제)_3박스(25%할인)    123750    3    1
           선택: 파이토레드큐민(정제)_6박스(36%할인)    422400   12    2
           선택: 퓨어레드큐민(분말)_3박스(46%할인)     316000   12    4
           퓨어레드큐민(분말)_12박스(55%할인)        267000   12    1
2025-04-16 선택: 파이토레드큐민(정제)_6박스(36%할인)    422400   12    2
           선택: 퓨어레드큐민(분말)_3박스(46%할인)     237000    9    3
           선택: 퓨어레드큐민(분말)_6박스(53%할인)     278000   12    2
총합                                      4400450  144   26

In [337]:
types = []

for i in temp_df['옵션명']:
    if '정제' in i:
        types.append('정제')
    elif '분말' in i:
        types.append('분말')

temp_df['제형'] = types

In [339]:
pivot_df3 = pd.pivot_table(temp_df, index=['주문일', '제형'], values=['매출', '박스수량'], aggfunc=['sum'], fill_value=0, margins=True, margins_name='총합')
pivot_df4 = pd.pivot_table(temp_df, index=['주문일', '제형'], values=['결제 번호'], aggfunc=['count'], fill_value=0, margins=True, margins_name='총합')

result_df2 = pd.DataFrame(pivot_df3)
result_df2[('sum', '주문수량')] = pivot_df4[('count', '결제 번호')]

result_df2.to_excel(f'/Users/seanyu_mac/Downloads/샤인약사님_매출_{datetime.now()}.xlsx')
result_df2

sum          
                    매출 박스수량 주문수량
주문일        제형                   
2025-04-14 분말   575000   18    4
           정제  1395900   42    6
2025-04-15 분말   583000   24    5
           정제   909150   27    4
2025-04-16 분말   515000   21    5
           정제   422400   12    2
총합             4400450  144   26

In [294]:
ka_df.index.size

31